<a href="https://www.kaggle.com/code/aleksandrchelyshkin/amazon-sales-sql-tableau?scriptVersionId=163403875" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

### This project explores data provided by the [E-Commerce Sales Dataset](https://www.kaggle.com/datasets/thedevastator/unlock-profits-with-e-commerce-sales-data) to answer key questions. 

### We will particularly investigate the "Amazon Sale Report.csv" using SQL Lite & Tableau.
### The viz is available [here](https://public.tableau.com/app/profile/aleksandr.chelyshkin/viz/Amazon_Sales_17082940665600/Dashboard1).

**Product Performance:**

* What are the top-selling product categories and items by quantity and revenue?

* How do order quantities vary, and what is the most common fulfilment method?

* Which items are frequently co-purchased, and can this inform product bundling and cross-selling strategies?

* What are the best-performing products monthly?

**Customer Behavior:**

* What are the return and cancellation rates?

* Are there any significant changes in the B2B vs. B2C mix?

**Geographical Distribution:**

* Which states and cities contribute the most to the sales volume?

In [1]:
import sqlite3
import pandas as pd

conn= sqlite3.connect("amazon_data.db")

In [2]:
file_path = "/kaggle/input/unlock-profits-with-e-commerce-sales-data/Amazon Sale Report.csv"

names = pd.read_csv(file_path, low_memory=False)

names.to_sql(
    'amazon_sales',  # Name of the SQL table
    conn,            # sqlite.Connection or sqlalchemy.engine.Engine
    if_exists='replace'  # Specifies how to behave if the table already exists
)

128975

Let's start with some basic queries to get an overview of the e-commerce dataset.

In [3]:
cursor = conn.cursor()
data_sql_1 = pd.read_sql("""
                        SELECT 
                            COUNT(*) as total
                        FROM amazon_sales 
                        ;
                        """,
                        conn)
print(data_sql_1)

    total
0  128975


In [4]:
#The total revenue is stated in INR   
data_sql_2 = pd.read_sql("""
                        SELECT 
                            SUM(amount) AS total_revenue 
                        FROM amazon_sales 
                        ;
                        """,
                        conn)
print(data_sql_2)

   total_revenue
0     78592678.3


In [5]:
data_sql_3 = pd.read_sql("""
                        SELECT 
                            Category, 
                            SUM(qty) AS total_quantity_sold
                        FROM amazon_sales
                        GROUP BY Category
                        ORDER BY total_quantity_sold DESC
                        ;
                        """,
                        conn)
print(data_sql_3)

        Category  total_quantity_sold
0            Set                45289
1          kurta                45045
2  Western Dress                13943
3            Top                 9903
4   Ethnic Dress                 1053
5         Blouse                  863
6         Bottom                  398
7          Saree                  152
8        Dupatta                    3


Here the query aggregates the data by product category, calculates the total quantity sold for each category + sorts the categories in descending order based on the total quantity sold.

Clearly, **Set and Kurta** are the top-selling product categories based on quantity sold, indicating strong consumer demand.

In [6]:
#Identifing the most common fulfilment methods used
data_sql_4 = pd.read_sql("""
                        WITH FulfillmentSummary AS (
                            SELECT 
                                Fulfilment, 
                                SUM(qty) AS total_quantity_sold
                            FROM amazon_sales
                            GROUP BY Fulfilment
                            )
                        SELECT 
                            fs.Fulfilment, 
                            fs.total_quantity_sold,
                            ROUND((fs.total_quantity_sold * 100.0 / total.total_qty), 2) AS percentage_sold
                        FROM 
                            FulfillmentSummary AS fs
                        CROSS JOIN (
                            SELECT SUM(qty) AS total_qty
                            FROM amazon_sales
                            ) AS total
                            ;
                            """,
                            conn)
print(data_sql_4)

  Fulfilment  total_quantity_sold  percentage_sold
0     Amazon                84087            72.09
1   Merchant                32562            27.91


Predictably, the majority of orders are handled by **Amazon (72.09%)** rather than by **Merchant (27.91%)**

Now let's check out the more detailed review.

In [7]:
data_sql_5 = pd.read_sql("""
                        SELECT 
                            CAST(SUBSTR(Date, 1, 2) AS INTEGER) AS month,
                            SUM(amount) AS total_revenue
                        FROM amazon_sales
                        GROUP BY month
                        ORDER BY total_revenue DESC
                        ;
                        """,
                        conn)
print(data_sql_5)

   month  total_revenue
0      4    28838708.32
1      5    26226476.75
2      6    23425809.38
3      3      101683.85


The **4th month** appears to be the most profitable in terms of revenue.

In [8]:
data_sql_6 = pd.read_sql("""
                        SELECT category, SUM(amount) AS total_sales_revenue
                        FROM amazon_sales
                        GROUP BY category
                        ORDER BY total_sales_revenue DESC
                        ;
                        """,
                        conn)
print(data_sql_6)

        Category  total_sales_revenue
0            Set          39204124.03
1          kurta          21299546.70
2  Western Dress          11216072.69
3            Top           5347792.30
4   Ethnic Dress            791217.66
5         Blouse            458408.18
6         Bottom            150667.98
7          Saree            123933.76
8        Dupatta               915.00


And the category **set** is the top-selling one followed by the **kurta**.

In [9]:
data_sql_7 = pd.read_sql("""
                        SELECT 
                            qty, 
                            COUNT(*) AS num_orders
                        FROM amazon_sales
                        GROUP BY qty
                        ORDER BY qty
                        ;
                        """,
                        conn)
print(data_sql_7)

   Qty  num_orders
0    0       12807
1    1      115780
2    2         341
3    3          32
4    4           9
5    5           2
6    8           1
7    9           1
8   13           1
9   15           1


* Here the output reveals that the most common order quantity in the dataset is 1, with 115,780 orders. 
* Then it followed by a substantial number of orders with 0 items (12,807 instances), possibly indicating canceled orders. 
* Order quantities beyond 1 show a sharp decline in frequency, with only a few cases of orders with 2 to 5 items, and rare occurrences of higher quantities (8, 9, 13, and 15 items).

In [10]:
data_sql_8 = pd.read_sql("""
                        SELECT
                            COUNT(DISTINCT `Order ID`) AS check_id,
                            COUNT(*) AS total_orders,
                            (COUNT(*) - COUNT(DISTINCT `Order ID`)) AS more_than_one_item
                        FROM amazon_sales
                        ;
                        """,
                        conn)
print(data_sql_8)

   check_id  total_orders  more_than_one_item
0    120378        128975                8597


This query computes the count of distinct order IDs, the total number of orders, which helps us to identify **orders with more than one item**. 

We should also check out the percentage.

In [11]:
data_sql_9 = pd.read_sql("""
                        WITH more_than_one_item AS (
                            SELECT 
                                COUNT(DISTINCT `Order ID`) AS check_id,
                                COUNT(*) AS total_orders,
                                (COUNT(*) - COUNT(DISTINCT `Order ID`)) AS orders_two_or_more
                            FROM amazon_sales
                            )
                        SELECT
                            ROUND((mti.orders_two_or_more * 100.0 / mti.total_orders), 2) AS percentage_orders
                        FROM more_than_one_item AS mti
                        CROSS JOIN (
                            SELECT COUNT(`Order ID`) AS total_ord
                            FROM amazon_sales 
                            ) AS total
                            ;
                            """,
                            conn)
print(data_sql_9)

   percentage_orders
0               6.67


The output of the query indicates that approximately **6.67%** of the orders in the amazon_sales dataset **contain more than one item**. 

This output suggests a relatively small portion of orders consist of multiple items.

In [12]:
data_sql_10 = pd.read_sql("""
                        SELECT
                            a.SKU AS item1,
                            b.SKU AS item2,
                            a.category,
                            b.category,
                            COUNT(*) AS num_times
                        FROM amazon_sales AS a
                        JOIN 
                            amazon_sales AS b ON a.`Order ID` = b.`Order ID` AND a.SKU < b.SKU
                        GROUP BY
                            a.SKU,
                            b.SKU
                        ORDER BY num_times DESC 
                        limit 10
                        ;
                        """,
                        conn)
print(data_sql_10)

               item1            item2       Category       Category  num_times
0    SET264-KR-NP-XL  SET268-KR-NP-XL            Set            Set         12
1      JNE3543-KR-XS  SET397-KR-NP-XS          kurta            Set          9
2       JNE3543-KR-S   SET397-KR-NP-S          kurta            Set          8
3       JNE3543-KR-S     JNE3787-KR-S          kurta          kurta          7
4       JNE3800-KR-S     JNE3801-KR-S  Western Dress          kurta          7
5  PJNE3252-KR-N-6XL  PJNE3291-KR-6XL          kurta          kurta          7
6     JNE3797-KR-A-M   SET324-KR-NP-M  Western Dress            Set          6
7       JNE3797-KR-L     JNE3801-KR-L  Western Dress          kurta          6
8      JNE3797-KR-XL    JNE3798-KR-XL  Western Dress  Western Dress          6
9     JNE3797-KR-XXL   JNE3801-KR-XXL  Western Dress          kurta          6


The result shows **pairs of speciffic items** +categories and the number of times they appear together across different orders. 

*This information usually provides valuable insights for product bundling and cross-selling.*

In [13]:
data_sql_11 = pd.read_sql("""
                        SELECT
                            month,
                            category,
                            size,
                            total_revenue
                        FROM 
                        (
                            SELECT 
                                month,
                                category,
                                size,
                                sum(amount) AS total_revenue,
                                ROW_NUMBER() OVER (PARTITION BY month ORDER BY sum(amount) DESC) AS rn
                        FROM 
                            (
                            SELECT
                                category,
                                size,
                                amount,
                                CAST(SUBSTR(Date, 1, 2) AS INTEGER) AS month
                            FROM amazon_sales
                            ) 
                        GROUP BY
                            category, size, month
                        ) as ranked_data
                        WHERE rn = 1
                        ORDER BY month
                            ;
                            """,
                            conn)
print(data_sql_11)

   month category size  total_revenue
0      3      Set    M       11995.00
1      4      Set    M     2923800.96
2      5      Set    M     2364132.76
3      6      Set    M     2004229.03


* The Set category with size "M" consistently emerges as the top-selling product across the months (March to June).

* There seems to be a trend of strong sales performance for this specific category and size combination throughout the analyzed period, with April recording exceptionally high revenue.

In [14]:
data_sql_12 = pd.read_sql("""
                       WITH CustomerBehavior AS (
                       SELECT
                            SUM(Qty) AS quantity_ordered,
                            COUNT(DISTINCT `Order ID`) AS orders_placed,
                            COUNT(DISTINCT CASE WHEN Status = 'Shipped - Delivered to Buyer' OR 'Shipped - Picked Up' THEN `Order ID` END) AS orders_delivered,
                            COUNT(DISTINCT CASE WHEN Status = 'Shipped - Returned to Seller' THEN `Order ID` END) AS orders_returned,
                            COUNT(DISTINCT CASE WHEN Status = 'Cancelled' THEN `Order ID` END) AS orders_cancelled,
                            SUM(CASE WHEN B2B = true THEN 1 ELSE 0 END) AS b2b_orders,
                            SUM(CASE WHEN B2B = false THEN 1 ELSE 0 END) AS b2c_orders
                        FROM amazon_sales
                        )
                        SELECT
                            quantity_ordered,
                            orders_placed,
                            orders_delivered,
                            orders_returned,
                            orders_cancelled,
                            b2b_orders,
                            b2c_orders,
                            CASE
                                WHEN orders_placed > 0 THEN (1.0 * orders_returned / orders_placed) * 100 
                                ELSE 0.0
                            END AS return_rate_percentage,
                            CASE
                                WHEN orders_placed > 0 THEN (1.0 * orders_cancelled / orders_placed) * 100
                                ELSE 0.0
                            END AS cancelled_rate_percentage
                        FROM CustomerBehavior
                        ;
                        """,
                        conn)
print(data_sql_12)

   quantity_ordered  orders_placed  orders_delivered  orders_returned  \
0            116649         120378             26566             1851   

   orders_cancelled  b2b_orders  b2c_orders  return_rate_percentage  \
0             17185         871      128104                1.537656   

   cancelled_rate_percentage  
0                  14.275864  


Looking at the details:

* The return rate is approximately 1.54%, indicating a relatively low return frequency, however, there's a substantial cancellation rate.

* The cancellation rate is notably higher at around 14.28%, suggesting a considerable proportion of orders are cancelled before fulfillment.

* B2C orders significantly outnumber B2B orders in the dataset, which is typical to e-commerce trends.


In [15]:
data_sql_13 = pd.read_sql("""
                        SELECT
                            `ship-state`,
                            sum(amount) as total_amount
                        FROM amazon_sales
                        GROUP BY 
                            `ship-state`
                        ORDER BY
                            total_amount DESC
                        LIMIT 10
                        ;
                        """,
                        conn)
print(data_sql_13)

       ship-state  total_amount
0     MAHARASHTRA   13335534.14
1       KARNATAKA   10481114.37
2       TELANGANA    6916615.65
3   UTTAR PRADESH    6816642.08
4      TAMIL NADU    6515650.11
5           DELHI    4235215.97
6          KERALA    3830227.58
7     WEST BENGAL    3507880.44
8  ANDHRA PRADESH    3219831.72
9         HARYANA    2882092.99


As for the geographic part, the feasible start would be to see states with high sales volumes in India.

* MAHARASHTRA: Emerges as the top performer with a total sales amount of $13,335,534.14, indicating significant economic activity and consumer demand within the state, particularly in cities like Mumbai.

* KARNATAKA: Follows closely behind Maharashtra with $10,481,114.37 in total sales. This suggests a robust market in Karnataka, primarily driven by cities like Bengaluru, known for its thriving tech industry and consumer market.

* TELANGANA and UTTAR PRADESH: Show substantial sales figures, indicating active commerce in Hyderabad and Ghaziabad, respectively. These regions contribute significantly to the overall sales landscape.

* TAMIL NADU, DELHI, and KERALA: Reflect notable sales volumes, underlining diverse economic activities and consumer preferences across Chennai, New Delhi, and Thiruvananthapuram.

* WEST BENGAL, ANDHRA PRADESH, and HARYANA: Contribute to the top states in terms of sales, signifying widespread consumer engagement and market opportunities within these regions.

In [16]:
data_sql_14 = pd.read_sql("""
                        SELECT
                            `ship-city`,
                            sum(amount) as total_amount
                        FROM amazon_sales
                        GROUP BY 
                            `ship-city`
                        ORDER BY
                            total_amount DESC
                        LIMIT 10
                        ;
                        """,
                        conn)
print(data_sql_14)

   ship-city  total_amount
0  BENGALURU    6849664.99
1  HYDERABAD    4946032.82
2     MUMBAI    3704461.80
3  NEW DELHI    3613874.78
4    CHENNAI    3098745.74
5       PUNE    2338518.18
6    KOLKATA    1414978.87
7   GURUGRAM    1221618.74
8      THANE    1004355.29
9    LUCKNOW     933926.34


By building a similar query, we also retrieve information about the top-performing cities based on total sales amounts

* BENGALURU: Leads the list with total sales amounting to $6,849,664.99, indicating significant economic activity and consumer spending in the city. This suggests Bengaluru's strong market presence and potential as a hub for commerce and business activities.

* HYDERABAD: Follows closely behind with total sales of $4,946,032.82, indicating a thriving market and consumer base in Hyderabad. The city's dynamic economy and diverse population contribute to its substantial sales figures.

* MUMBAI: With $3,704,461.80 in total sales, Mumbai ranks third on the list. As a major financial and commercial center, Mumbai's sales performance reflects its status as a key player in India's economy.

* NEW DELHI: Shows significant sales figures, totalling $3,613,874.78, highlighting the city's importance as a major consumer market and economic hub.

* CHENNAI, PUNE, KOLKATA, GURUGRAM, THANE, and LUCKNOW: Also contribute to the top cities in terms of sales, reflecting diverse economic activities and consumer preferences across these regions.